In [ ]:
import pandas as pd
from time import sleep
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

def f_score(cm, index = 2, beta = 1):
    tp = cm[index][index]
    fp = cm[:, index].sum() - cm[index][index]
    relevant = cm[index, :].sum()
    precision = tp/(tp + fp)
    recall = tp / (relevant)
    
    return (1 + beta ** 2) * (precision * recall)/(beta * beta * precision + recall)

    
    
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/uniform_sample/results/9-13-and-8-28-with-confidence.csv'
df = pd.read_csv(path)

manual_column_name = 'alex'

if (manual_column_name not in set(df.columns)):
    df[manual_column_name] = [None for _ in range(len(df))]

options = {str(idx): el for idx, el in enumerate(['context', 'uses', 'extends'])} | {'*': '*'}
print(options)

df = df[df['strippedModelKey'] != 'ase']
labeling_on = False

In [ ]:
mask = df[manual_column_name].isna()
print(f"{mask.sum()}/{len(mask)} left.", flush = True)

In [ ]:
df['classification_mod'] = [c if correct_attribution >= .3 else 'context' for c, correct_attribution in zip(df['classification'], df['gpt4o_prompt1_10'])]

In [ ]:
import random
from IPython.display import clear_output


mask = df[manual_column_name].isna()
while mask.sum() > 0 and labeling_on:
    idx = random.choice(df[mask].index)
    
    print(f"{mask.sum()}/{len(mask)} left.", flush = True)
    print(options, flush = True)
    print(f"Model Key:{df.at[idx,'modelKey']}", flush = True)
    print(df.at[idx, 'multisentence'], flush = True)
    
    sleep(1)

    response = input()
    
    if (response == 'exit'):
        #df.to_csv(path) commenting out to avoid oopsies
        clear_output()
        break
    
    df.loc[idx, manual_column_name] = options.get(response)
    mask = df[manual_column_name].isna()
    clear_output()

    

In [ ]:
not_table = lambda s: s.find('\\begin{tab') == -1

confidence_threshold = 5
subset_mask = (df['alex'].notna() & df['multisentence'].apply(not_table)
               #& (df['alex_confidence'] > confidence_threshold)
               #& (df['alex_confidence'] <= confidence_threshold)\
               )

df_not_null = df[subset_mask]

true, pred = df_not_null['alex'], df_not_null['classification_mod']

print(Counter(true), Counter(pred))

cm = confusion_matrix(true, pred)

use_weights = False
if use_weights:
    weights = np.array([0.855154, 0.016340, 0.128506]).reshape(1, 3)
    cm  = cm * weights 
    cm = cm * (1000 / cm.sum())
    cm = np.round(cm).astype(int)
    assert(np.abs(cm.sum() - 1000) < 5), cm.sum()
    
print(cm.sum())

y_label, x_label = 'alex'


#labels = ['context', 'extends', 'uses']
labels = ['context', 'not_context']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, cbar = False, annot_kws={"size": 20})
plt.xlabel(x_label)
plt.ylabel(y_label)


#print(f"Uses F1: {f_score(cm, index = 2, beta = 1)}")
#print(f"Extends F1: {f_score(cm, index = 1)}")